In [15]:
from qiskit import Aer
from qiskit.aqua.algorithms import VQE, ExactEigensolver, QAOA
from qiskit.aqua.components.optimizers import SPSA
from qiskit.aqua import QuantumInstance
from qiskit.aqua import aqua_globals
from qiskit.circuit.library import RealAmplitudes
from qiskit.visualization import circuit_visualization
from docplex.mp.model import Model
from qiskit.optimization.applications.ising import docplex
from qiskit import IBMQ
# IBMQ.load_account()
import numpy as np

from collections import OrderedDict

/home/julia/.local/lib/python3.6/site-packages/qiskit/providers/ibmq/ibmqfactory.py:192: UserWarning: Timestamps in IBMQ backend properties, jobs, and job results are all now in local time instead of UTC.
  warnings.warn('Timestamps in IBMQ backend properties, jobs, and job results '
ibmqfactory.load_account:WARNING:2021-02-11 21:25:09,903: Credentials are already in use. The existing account in the session will be replaced.


In [16]:
M = [2,6,3]
K = [1,4,2]
T = [12, 6, 24]
d = 19

def get_time_matrix(M, T):
    r = []
    for i in M:
        tmp = []
        for j in T:
            tmp.append(j / i)
        r.append(tmp)
    return np.array(r)


def get_cost_matrix(time_matrix, K):
    m = []
    for i in range(len(time_matrix)):
        tmp = []
        for j in time_matrix[i]:
            tmp.append(K[i] * j)
        m.append(tmp)
    return m

time_matrix = np.array(get_time_matrix(M, T))
cost_matrix = np.array(get_cost_matrix(time_matrix, K))

print("Time matrix")
print(time_matrix)
print("\nCost matrix")
print(cost_matrix)

Time matrix
[[ 6.  3. 12.]
 [ 2.  1.  4.]
 [ 4.  2.  8.]]

Cost matrix
[[ 6.  3. 12.]
 [ 8.  4. 16.]
 [ 8.  4. 16.]]


In [17]:
def sample_most_likely(state_vector):
    if isinstance(state_vector, (OrderedDict, dict)):
        # get the binary string with the largest count
        binary_string = sorted(state_vector.items(), key=lambda kv: kv[1])
        print(binary_string)
        repetitions = int(binary_string[-1][1])
        binary_string = binary_string[-1][0]
        x = np.asarray([int(y) for y in reversed(list(binary_string))])
        return x, repetitions
    return [], 0

optimal_key = "0000001000"
time_limit = 19

def get_stats_for_result(dict_res):
    optimal = 0
    correct = 0
    incorrect = 0
    correct_config = 0
    incorrect_config = 0
     
    if optimal_key in dict_res:
        optimal = dict_res[optimal_key]
    for key, val in dict_res.items():
        if is_correct(key):
            correct += val
            correct_config += 1
        else:
            incorrect += val
            incorrect_config += 1
            
    print('most likely solution: ',sample_most_likely(dict_res))      
    print("optimal: ", optimal)
    print("correct solutions: ", correct)
    print("incorrect solutions: ", incorrect)
    print("correct configs: ", correct_config)
    print("incorrect configs: ", incorrect_config)

    
def is_correct(key):
    reversed_key = key[::-1]
    if solution_vector_correct(reversed_key) and execution_time(reversed_key) <= time_limit:
        return True
    return False

correct_machines = ['00', '01', '11']
machine_to_index = {'00': 0, '01': 1, '11': 2}

def execution_time(k):
    slack_sum = int(k[6]) * 8 + int(k[7]) * 4 + int(k[8]) * 2 + int(k[9]) * 1
    
    task1_machine = machine_to_index.get(k[0:2])
    task2_machine = machine_to_index.get(k[2:4])
    task3_machine = machine_to_index.get(k[4:6])
    
    # for vector "10" (the only incorrect one), assign sum of all machines times for task
    task_time_sum =  np.sum(time_matrix, axis=0) 
    task1_time = task_time_sum[0] if task1_machine == None else time_matrix[task1_machine, 0]
    task2_time = task_time_sum[1] if task2_machine == None else time_matrix[task2_machine, 1]
    task3_time = task_time_sum[2] if task3_machine == None else time_matrix[task3_machine, 2]
    
    return slack_sum + task1_time + task2_time + task3_time
    
def solution_vector_correct(vector):
    task1_machine = vector[0:2]
    task2_machine = vector[2:4]
    task3_machine = vector[4:6]

    if task1_machine in correct_machines and task2_machine in correct_machines and task3_machine in correct_machines:
        return True
    return False

In [30]:
def getMdl1(A):
    mdl = Model(name='cost')
    x = {(i, j) : mdl.binary_var(name='x_{0}_{1}'.format(i, j)) for i in range(0, 3) for j in range(0, 2)}
    x.update({i : mdl.binary_var(name='x_{0}'.format(i)) for i in range(7, 11)})
    objective = A * mdl.sum(
          cost_matrix[2, i] * (1 - x[(i, 0)]) * (1 - x[(i, 1)])
        + cost_matrix[1, i] * (1 - x[(i, 0)]) * x[(i, 1)]
        + cost_matrix[0, i] * x[(i, 0)] * (1 - x[(i, 1)])
        for i in range(0, 3)
    )
    #why adding (x[7] + x[8] + x[9] + x[10])?
    mdl.minimize(objective)
    return mdl

def getMdl2(B):
    mdl = Model(name='machine-usage')
    x = {(i, j) : mdl.binary_var(name='x_{0}_{1}'.format(i, j)) for i in range(0, 3) for j in range(0, 2)}
    x.update({i : mdl.binary_var(name='x_{0}'.format(i)) for i in range(7, 11)})
    objective = B * mdl.sum(x[(i, 0)] * x[(i, 1)] for i in range(0, 3))
    mdl.minimize(objective)
    return mdl

def getMdl3(C):
    mdl = Model(name='deadline')
    x = {(i, j) : mdl.binary_var(name='x_{0}_{1}'.format(i, j)) for i in range(0, 3) for j in range(0, 2)}
    x.update({i : mdl.binary_var(name='x_{0}'.format(i)) for i in range(7, 11)})
    objective = C * (d - mdl.sum(
        [
              time_matrix[2, i] * (1 - x[(i, 0)]) * (1 - x[(i, 1)])
            + time_matrix[1, i] * (1 - x[(i, 0)]) * x[(i, 1)]
            + time_matrix[0, i] * x[(i, 0)] * (1 - x[(i, 1)])
            for i in range(0, 3)
        ]
        + [8 * x[7] + 4 * x[8] + 2 * x[9] + x[10]]
    )) ** 2
    mdl.minimize(objective)
    return mdl

In [43]:
time_matrix[:, 2]

array([12.,  4.,  8.])

In [31]:
from docplex.mp.model import Model
from qiskit.optimization.problems import QuadraticProgram

def createOp(A, B, C):
    # cost
    mdl1 = getMdl1(A)
    qp1 = QuadraticProgram()
    qp1.from_docplex(mdl1)
    H1, _ = qp1.to_ising()

    # machine usage
    mdl2 = getMdl2(B)
    qp2 = QuadraticProgram()
    qp2.from_docplex(mdl2)
    H2, _ = qp2.to_ising()

    # deadline
    mdl3 = getMdl3(C)
    qp3 = QuadraticProgram()
    qp3.from_docplex(mdl3)
    H3, _ = qp3.to_ising()
    
    return H1 + H2 + H3

#     print(qp3.export_as_lp_string())

In [32]:
seed = 123569 
aqua_globals.random_seed = seed
entanglement='full' #test as well 'circular', 'sca', 'linear'

A = 1
B = 20
C = 1

reps = 2
max_trials = 1000
shots = 100

qubitOp = createOp(A, B, C)

def compute_eigenvalues():
    from qiskit.aqua.algorithms import NumPyEigensolver
    count = 5
    eigensolver = NumPyEigensolver(qubitOp, count)
    eigensolver_result = eigensolver.compute_eigenvalues()
    print(eigensolver_result.eigenvalues)
    for eigenstate in eigensolver_result.eigenstates:
        print(eigenstate.sample())

compute_eigenvalues()

spsa = SPSA(maxiter=max_trials)
ry = RealAmplitudes(qubitOp.num_qubits, reps=reps, entanglement=entanglement)
vqe = VQE(qubitOp, ry, spsa)


#p = 2
#qaoa = QAOA(qubitOp, spsa, p)

# provider= IBMQ.get_provider('ibm-q')
backend = Aer.get_backend('qasm_simulator')
quantum_instance = QuantumInstance(backend,seed_simulator=seed, seed_transpiler=seed, shots=shots)
result = vqe.run(quantum_instance)   
state_vector = result['eigenstate']
get_stats_for_result(state_vector)

DOcplexException: Cannot take the square of a quadratic term: 4x_0_0*x_0_1+2x_1_0*x_1_1+8x_2_0*x_2_1-2x_0_0+2x_0_1-x_1_0+x_1_1-4x_2_0+4x_2_1-8x_7-4x_8-2x_9-x_10+5

In [33]:
vqe.get_optimal_cost()
#qaoa.get_optimal_cost()

NameError: name 'vqe' is not defined

In [ ]:
vqe.get_optimal_vector()
#qaoa.get_optimal_vector()